In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

In [32]:

# API Endpoint
url = "https://stats-api.dln.trade/api/Orders/filteredList"

# Function to fetch all orders
def fetch_all_orders(ite):
    all_orders = []
    skip = 0
    take = 100  # Adjust based on API limits
    df_final = pd.DataFrame()
    for i in tqdm(range(ite)):
        payload = {"skip": skip, "take": take}
        response = requests.post(url, json=payload)
        
        if response.status_code == 200:
            orders = pd.DataFrame(response.json())
            df_final = pd.concat([df_final, orders])
            skip += take
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break
    try: 
        df = pd.json_normalize(df_final['orders'])
        return df[[
                    'creationTimestamp', 'state', 'externalCallState', 'orderId.stringValue',
                    'giveOfferWithMetadata.chainId.stringValue', 'giveOfferWithMetadata.tokenAddress.stringValue',
                    'giveOfferWithMetadata.amount.stringValue', 'giveOfferWithMetadata.finalAmount.stringValue',
                    'giveOfferWithMetadata.metadata.name', 'giveOfferWithMetadata.metadata.symbol',
                    'giveOfferWithMetadata.metadata.logoURI', 'takeOfferWithMetadata.chainId.stringValue',
                    'takeOfferWithMetadata.tokenAddress.stringValue', 'takeOfferWithMetadata.amount.stringValue',
                    'takeOfferWithMetadata.finalAmount.stringValue', 'takeOfferWithMetadata.metadata.name',
                    'takeOfferWithMetadata.metadata.symbol', 'takeOfferWithMetadata.metadata.logoURI',
                    'finalPercentFee.stringValue', 'fixFee.stringValue',
                    'affiliateFee.beneficiarySrc.stringValue', 'affiliateFee.amount.stringValue',
                    'unlockAuthorityDst.stringValue', 'createEventTransactionHash.stringValue',
                    'orderMetadata.version', 'orderMetadata.creationProcessType',
                    'orderMetadata.origin', 'orderMetadata.additionalTakerIncentiveBps',
                    'orderMetadata.operatingExpensesAmount', 'orderMetadata.recommendedTakeAmount',
                    'orderMetadata.metadata','orderMetadata.promotionType'
                    ]]

    except Exception as e : 
        print('error normialising dataframe')
        print(e)
        return df_final['orders']

In [39]:
df = fetch_all_orders(10)
df['timestamp'] = pd.to_datetime(df['creationTimestamp'], unit = 's')


filtered_df = df[[
                'timestamp', 'creationTimestamp',
                'giveOfferWithMetadata.metadata.symbol', 
                'giveOfferWithMetadata.amount.stringValue',
                'giveOfferWithMetadata.finalAmount.stringValue',
                'takeOfferWithMetadata.metadata.symbol',
                'takeOfferWithMetadata.amount.stringValue',
                'finalPercentFee.stringValue']]

print('pct of orders with different tokens:' , len(filtered_df.loc[filtered_df['giveOfferWithMetadata.metadata.symbol'] != filtered_df['takeOfferWithMetadata.metadata.symbol']])/len(filtered_df))
filtered_df.loc[filtered_df['giveOfferWithMetadata.metadata.symbol'] != filtered_df['takeOfferWithMetadata.metadata.symbol']]

100%|██████████| 10/10 [00:05<00:00,  1.72it/s]

pct of orders with different tokens: 0.633


,timestamp,creationTimestamp,giveOfferWithMetadata.metadata.symbol,giveOfferWithMetadata.amount.stringValue,giveOfferWithMetadata.finalAmount.stringValue,takeOfferWithMetadata.metadata.symbol,takeOfferWithMetadata.amount.stringValue,finalPercentFee.stringValue
3,2025-02-15 11:27:38,1739618858,USDC,71371873151701145297,71371873151701145297,1INCH,239326001283376767014,28560173330012463
4,2025-02-15 11:27:26,1739618846,WETH,2408171159369265,2408171159369265,USDC,5342592,963653925317
5,2025-02-15 11:27:17,1739618837,USDC,187588503,187588503,BNB,278937554941730945,75065
6,2025-02-15 11:27:09,1739618829,USDC,38438507,38438507,SOL,187036133,15396
7,2025-02-15 11:27:08,1739618828,ETH,8591467540396441,8591467540396441,BNB,33646399061165783,3437962201038
...,...,...,...,...,...,...,...,...
989,2025-02-15 05:29:28,1739597368,USDC,109152225,109152225,ETH,39522510527817229,43678
990,2025-02-15 05:29:05,1739597345,USDC,32593629783506604416,32593629783506604416,SOL,154456557,13055729935312078
992,2025-02-15 05:28:07,1739597287,USDC,40115666,40115666,S,70729740380112659484,16052
994,2025-02-15 05:27:49,1739597269,USDC,1006124707,1006124707,USDT,1004799318451172722385,403620
